In [1]:
import pandas as pd
import re
from collections import Counter
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pytz

show cout 2026

In [2]:
# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [3]:

data['date'] = pd.to_datetime(data['date'])


In [4]:

def convert_utc_to_cet(df, date_column='date'):
    """
    Convert UTC timestamps to Central European Time (CET/CEST) with proper DST handling
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the date column
    date_column (str): Name of the column containing UTC timestamps
    
    Returns:
    pd.DataFrame: DataFrame with converted timestamps
    """
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Ensure timestamps are UTC aware
    if df[date_column].dt.tz is None:
        df[date_column] = df[date_column].dt.tz_localize('UTC')
    elif df[date_column].dt.tz != pytz.UTC:
        df[date_column] = df[date_column].dt.tz_convert('UTC')
    
    # Convert to CET/CEST (Europe/Berlin includes proper DST handling)
    df[date_column] = df[date_column].dt.tz_convert('Europe/Berlin')
    
    return df



In [5]:
data = convert_utc_to_cet(data)

In [6]:
data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("CHILI_POE", "chili_con_bacon")
data["user"] = data["user"].replace("Chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("trek_x", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uwu_cougar", "uuccugr")
data["user"] = data["user"].replace("uuccugr_","uuccugr")
data["user"] = data["user"].replace("StanIV4_", "stan_iv4")
data["user"] = data["user"].replace("Muuskie2", "Muuskie")
data["user"] = data["user"].replace("nishad_more1311", "nishad13")
data["user"] = data["user"].replace("softarballt", "softarr")
data["user"] = data["user"].replace("softarballtt23", "softarr")
data["user"] = data["user"].replace("lajosbarnabas", "lajoss__")
data["user"] = data["user"].replace("Bonkwiththefunk", "bonk67")
data["user"] = data["user"].replace("qfishyy11", "bonk67")

In [7]:

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}


In [8]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [9]:
# 7tv list
tv7_emotes = """omeFaded nonono wideSpeedFear wideSpeedDesert SpeedOmg Disappointed ME? ome13 doroPfft CONFUSED doro52 wideSpeedConcerned doroBANGER doroVibe DoroBrainPLug NAHHHH intrigued oda WOOF BANG peepoFirework Gladge omeMoji SobBounce belka doroHips dorochan doroXi DOMEYES DOWAY FRENCH YeahIGetIt doroPls Offline sheMightBeRight 89 dwerk wideGkeyDance3 76 xqcCheer omeCheer Dmile zoro LMAO iguessbro doroScary haii SON omeBru wideSpeedCall plink laracroft ADHD yipe doroWaiting 67 FOWL doroLOL OfCourse tyler67 THYME peepoLost speed61 GOODMOOD Jercula ILOVECS CLOWN thatsCrazy xqcBleh omEING wideSpeedLaugh15 WHATTHEHELI RickyDicky jonkler wideSpeedPumpkin wideSpeedLaugh16 SPIDER chillCat LEEKED soblaugh BANANA wideSpeedLaugh21 Binoculars footstepmenace ome99 doroBanger GhankYou NO YES auntiePls OMEGADANCEBUTGHAST gkeyRide GENERATINGGODSEED perimeter sideeye omePfft MySunshine posture ??? twocatsfightingonacouch dSmile wideSpeedLaugh4 flirtt Suspicion zigzag YeatCat nom omeSad shomonting thinking agafat FirstTimeClanka DORVIS gasp RISING dd uncPLS domgBruh Deafge OMEGADANCEBUTFASTER kim3 Buggin speed8 .... d32 ome54 ohok minionBike Clown Explosion hackingCD JermaSoy MathTime MoneyRain PokiShare TakingNotes :0 :3 :33 :tf: !boost +1 0pixel 1DLove 3Head 3Heading 4House 4Shrug 4Weird 5Head AAAA Acknowledged ACTINUP ADHD agahi AIM AIRBALL AIWITHTHEBRAIDS Alarm ALE Alfred AlienDance AlienPls AlienPls2 AlienPls3 Aloo Alright amongE ANGRE ANOTHERONE Ant AREYOUAGIRL AREYOUAGIRLFtxQcYellingAtYou areyoufr AreYouSeriousRightNeow arnoldHalt arthur Assept AURA AwHellNah Aware AWOO AWOOGA axelF ayo bah BAND Banger BANGER banties Barack barryArrive Barry63 BantiesPaulBeef Based BASED BatChestAbove batman Batman batJAM batPls Beatles Bedge BEG BEGGING Bello BigD bieberDougie BELIEVERS BibleThump BINGO Bleh Bloons BOOBA bog BOOM BOOMIES BOINK BORGIR Borfday brbToilet Broadcaster brb Bruh BRUHMM bruv buh buhbye buhFlipExplode BUSSIN BUSSERS bye CanIHaveADollar cannySilly catAsk catBusiness CatCozy catDespair catEat catJAM catKiss catPls catSigh catSmash CatTime catTwerk CAUGHT Caught CaughtIn4K Celebrating CHADDING characterSelected CHATTERS chatting cheerleaders ChillGuy Chillin Cinema chilling clappi Clap classic CLEAN Clueless CLIPPERS CLOWNDETECTED COCKA cokeBreak COMEHERE Concerned Considering Cooked COPIUM crabPls Crunch CS2 Cuck Cuh D: damily Damn dansi dash Dave deadassFaint Delusional DemonTiming Dentge despair DespairRyan Devious DIESOFCRINGE Dime DinkDonk Dinema doggoSlava dogJAM DogLookingWickedAndCool doid dojaPls dome44 dome32 donowall donoWall doroAunt doroBleh dorobubu doroCD DoroCheer doroFiddy doroFlex doroGHOST doroHEAD doroKick doroL doroMAD doroPray doroRip doroSoy DoroTalkingAgain Dorozea doster DOUBTERS DRAIN Drake DRAMA dreamwastaken drooling drukiDnace drukiDnace2 duaKiss dudWhat EDGE EDM EDITING emo erinNya essaying ewphop eww EZ EZdodge Exerpas Explosion eyeroll fadedthanaho FARMING FeelsBadMan FeelsDankMan FeelsBlackScreen FeelsGladMan FeelsLagMan FeelsLateMan FeelsOkayMan FeelsStrongMan FeelsTiredMan FeelsWeirdMan FeelsWowMan fein FEINFEINFEINFEINFEINFEINFEINFEI FellOff fembajJAM Fiddy FiddyWtf FINALLY firewriting FirstTime FirstTimeBackseating FirstTimeChadder FirstTimeChatter FirstTimeEmoteFail FirstTimeGooner FirstTimePepega FirstTimeTest firsttimebuh FLASHBANG flightnotL Flirt Flushed fnaf footstep forsenCD forsenLaughingAtYou ForsenSingingAtYou forsenPls fortnite fr freakbob freakyfredday freddy Freedom FUNNY g32 GAGAGA gachiGASM gachiHYPER gamily GAMBA GameplayTime GAMING GatieG Gaught GENIUS GetALoadOfThisGuy gg GIGACHAD GIGACHAIR GIGACLAUS GIGAMODS GIGAMOD gigl gkeyFlip gkeyPregnantBounce gkeySMP gkeyUwu gkeywide gkeyWiding gkitten GivenUp girlBoss gkitten glorpaga glorpdetective glorp GlorpMeeting glorprave gmoney goaler goat goblin44 GODDID Gogging GoodBye Gooner gooner GoodTake GOONING GotCaughtTrolling GotEEM gothKiss gPls greetingsladies GREEDY GROOTING GRRR GULP GuitarTime GYAT HABIBI hackingCD HACKERMANS hai HAH HaltEinfachDeineFresseDuHurensohn HandsUp happi HARAM HarryStylesKiss Headbang healed HECOOKING heh HEHE HEHEHEHA Heisenberj HELLO HELP helvete Herewego hesRight heyywithrizz HEYYY hi hiii hiiii Hmm HOBBY HOLY HolyFuck homelessPOV HowDoWeTellHer HowDoWeTellHim hue HUH HUHHHHHHHHHH iAsked ICANT idiot iDrive IFISPEAK IfYouCantSeeThisEmoteUseExclamationMark7tv Ignored IGON imback IMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGET ImNotOk ImtiredBoss INTENSEGAMING islandboy ISeeYou itsover itstime Jackass jacob1 jacob2 jacob3 jacob4 jah Jammies JARVIS Jay JermaSoy jiggy job JOB Joel joever john Johnporkiscalling JokerHAHA JokerLaugh juh JumpScared JUMPSCARE JustAChillGuy JustAnotherDay JustHowItIs justinbieber KaiCenatOhiogyatwithskibiditoiletwatchingtheWrizzhappeningrightinfrontofhimwithfanumtaxtaxingthegyat KanyeStare KEKW KENOUGH KeyShaker kim3 kittyBANGER kittyBop KKalinka KKonaW KKool kratos Lamonting LastTimeChatter lava lebronArrive lebronJAM lebronTROLL LEBRONNN lemon Lemon LetsBingo LETHERCOOK LETSFUCKINGJOE LETSGO LieMeter life Life Listening LiterallyMe Lithuanian LittleTrolling LiveReaction LL LMAOFREAKY lmao Loading LOCKIN lockedin LOL Looking LookUp lore luh lurkk luton LULE LULW MAJ Madge ManchesterUnited Massive? MarblesTime Martin matSad maxwin MeRN me: MeWhenIBuyEgyptianProperty MEGALUL mee Memories merch mhm MicTime mikuPls mindloud modCheck ModAbuse MODDING Modding mods MODS Mog monakS monday monkeyListening monkeySip MONKA monkaTOS monkaW MONKE MinionHoting MoneyRain muted mutted MUGA MVP MVPOfFarallah MYHEARTILOVEDHER myIQ MYLIFE NAILS NAILSING NAHH NAHHH NAHHHH nananAYAYA NAUR NAvsEU Nerd niceguy NOCHECKMARKS NODDERS NOIDONTTHINKSO NoMaidens NOOPERS NOOOOO NOHORNY noonecares NOSHOT NOTED notListening notxqcL NOW NOWAY NOWAYING np nt nuhuh nyehehehe nyanPls nya o7 Ogre ohhh ohhhhhhhhh OHMYGAWDD ohneFinger ohno ohSHIT oj OK Okei okak OLDWORK OM om omE ome10 ome101 ome104 ome105 ome14 ome15 ome18 ome21 ome29 ome32 ome4 ome41 ome44 ome44444444 ome47 ome5 ome51 ome52 ome55 ome57 ome67 ome69 ome79 ome808 ome83 ome9 ome90 ome96 ome99 OMEGADANCE OMEGALUL OMEGALULiguess omEE omeJAM omeJudging omeOhSHIT omeScrajj omeStare OMEYES omeWiggle OMFG omgBruh ongang OneGuy ONEMORE OnMyWayToDoroMomHouse OOOO oopsie otag OuttaPocket OVERWATCH OVERWORKING OverwhelminglyWholesome owoCheer PagBounce PagChomp PagMan Panam parasocial Parasocial PARASOCIAL paris paul Paul paulNya PauseMan PAUSENEMOGU Peace PEEPEES peepoAds peepoBox peepoBelievers peepoClap peepoComfy peepoDJ peepoDoubters peepoEvil peepoFarmer peepoFat peepoGiggles peepoHappy peepoHey peepoHug peepoKiss peepoLeave peepoLegs peepoLove peepoMarch peepoPls peepoPride peepoRiot peepoSad peepoShy peepoSmile peepoStop peepoTalk pepeAgony pepeGun PepeHands pepeJAM PepeLaugh pepePoint PepePls pepeW Pepega PepegaAim PepegaChat PepegaReading PepoG Petter Pffttt Pffttt2 phew phpk pickle PianoTime Pipege pKitten pL Please pleading plink-182 plinkVibe plonk pmo Plotge PogO PogU pointless pokiFlirt pol POLICE Pondering popipopipipopipo poroPls POVbornbefore2000 ppHop ppL ppOverheat Prayge prePffttt PRIMERS PTSD pulNya PuzzleTime qq ragebait RAGEY RAHH RainTime RAMBOLMG RareParrot ratomilton RaveDance RaveTime ratio Reacting RealForsen ReallyMad RebeccaBlack Reddit RememberTheDays RibertJam RiddleMeThis RIPBOZO RIRI Rizzler RobertJam ROFL RoxyPotato RUNNING rt ryanArrive Sadding Sadge SADge SAJ SAVEME SCATTER saythatagain scawy SCHEISSE SCHIZO SCRAPETHATSHITJOHNNY SCHTOP sdd SERIOUSLY SEXO shogaNya Shits shutup Shruge silliness sisyphus Sippin Sits skip SLAY Sleepo Smile smh Smoge SmurfHey Smurfing SNACKING SNEAK SNIFFA sob SOLARFLARE songbird sotruebestie SOYSCREAM Speechless speed1 speed2 speed21 speed25 speed32 speed4 speed44 speed8 speed88 speedVibe spfLEAN:()wiltee_()tonyhawkproskater4:-:-:_FREEWAVE3-encinoman--:enteringwalmart:-wheezethelean-123 SpeedLaugh SpeedLeft SpeedR spongePls squadHips Stare Staring steve Steve SteerR StreamEnding STREAMER STREAMERSGIVINGTHEWORSTFUCKINGTAKESINEXISTENCE StoryTime Surfing SurE sus susDog Susge SUSSY Swag swagJAM ta tak TakingNotes TeamEDWARD test THATHIT ThatsJustMe ThePaulers TheVoices TheWolfInMe Thinking Thinking2 ThisChat ThisIsMinecraft TIMEOUT Tomfoolery totallylistening TriJam TriKool TriSad TRUEING TRIVSsorry ts Tuckge tuff TWEAK typeshit typhu UGH um UltraMad unibrow unemployment unmod uwu uuh VALORANT veryDoro VeryKey VeryPog VeryPogftxQcInTheShower vibePls VibePls VIDEOGAME VIEWERS vips Voices wade Waddup waga wah waiting Waiting WAHHH WAJAJA WAIT WAITWAITWAIT WakeTheFuckUpSamuraiWeHaveACityToBurn wallE waltuh walterShocked WalterVibe War WasZumPenis WATAFUCKEDUPDAY WatchingStream WAYTOODANK wdym WeAreLive WeDoNotCare WEDIDIT WEEWOO WeGood WePaid WHATAFUCKEDUPDAY WHAT WHATTT wheresmyhug Whenyourinnerwolfreleases WideAlERT WideCatGroove wideDvaAss WideHardo WidelebronJAM widemonkaGIGAftRobertDowneyJr wideprespeedlaugh WideRaveTime wideReacting wideSpeedLaugh3 widetime WidezyzzPls wig WineTime winton Wisdom woah Wokege WOT wot wrapitup WW wowie Xd xar2EDM xdd XDoubt xJAM xqc32 xqcBOZO xqcDespair xqcFuel xqcGoofy xqcL xqcSCHIZO xqcSlam xqcTake xqcTwerk xQcVeryWide YAAAY YamesBond YANITED YAPPING YeahThatsWhatIWouldaDid YEAHHH YEP YESS YIPIEE yonose Yoink YOOLOOKATTHISCATDOINITSLILDANCYDANCEINTOABREAKDANCEMOVE Yooo YOUDIED YouGotMe YouWouldntGetIt ZAMN ZhongXina zyzzBass zyzzJAM"""

# Convert to list and clean 
emote_list = [emote.strip() for emote in tv7_emotes.split() if emote.strip()]

# Remove any remaining duplicates (if any)
unique_emotes = sorted(list(set(emote_list)))

print(f"Total unique emotes: {len(unique_emotes)}")

# Create the final shortened list
final_emote_list = unique_emotes

Total unique emotes: 1040


2026 Starts here

In [10]:
new_emotes = [
    'ome62','Staredown','PentagramOfFarallah','Gloving','footgammaRadiation',
    'RAAAAAAAAGH','praise','doroRage','AndreSmithing','omePIECE','Rime',
    'doroPIECE','kaiReading','Deadge','furi','blub','duh','nuh','gopissgirl',
    'plong','Awesome','o','feaky','DoroThinking','evol','doro18','staycalm',
    'omeFOWL','s','fairs','deal','moshimoshi','gn'
]

final_emote_list = list(set(final_emote_list) | set(new_emotes))


In [11]:
import pandas as pd
from collections import Counter

# Filter only January 2026
jan_2026 = data[(data["date"].dt.year == 2026) & (data["date"].dt.month == 1)]

# Find the first message date for each user
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message was in January 2026
new_chatters = first_messages[
    (first_messages["date"].dt.year == 2026) & (first_messages["date"].dt.month == 1)
]

# Get the number of new chatters
num_new_chatters = new_chatters["user"].nunique()

print(f"Number of new chatters in Jan 2026: {num_new_chatters}")

stream_counts = jan_2026['stream'].value_counts().reset_index()
print(f"Number of Streams in Jan 2026: {len(stream_counts)}")
print(f"Number of Messages in Jan 2026: {jan_2026.shape[0]}")
print(f"Number of Users in Jan 2026: {jan_2026['user'].nunique()}")

# Count number of messages per user in January 2026
user_counts = jan_2026['user'].value_counts().reset_index()

# Rename columns for clarity
user_counts.columns = ['user', 'message_count']

# Get the top 10 users
top_10_users = user_counts.head(10)
print("\nTop 10 users in Jan 2026:")
print(top_10_users)

# --- Emote Analysis ---
emote_counter = Counter()

# Go through each message and count emotes
for message in jan_2026["message"]:
    words = message.split()
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

# Get top 10 emotes
top_10_emotes = emote_counter.most_common(10)

print("\nTop 10 emotes in Jan 2026:")
for emote, count in top_10_emotes:
    print(f"{emote}: {count} times")

# --- Activity Spikes ---
# Make a copy to avoid SettingWithCopyWarning
jan_2026 = jan_2026.copy()

# Round timestamps to nearest 5-minute interval
jan_2026["5min"] = jan_2026["date"].dt.floor("5min")

# Count messages per 5-minute interval
message_counts = jan_2026.groupby("5min").size().reset_index(name="message_count")

# Get top 5 busiest 5-minute intervals
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("\nTop 5 busiest 5-minute intervals in Jan 2026:")
print(top_5_fastest)

# --- Stream Velocity ---
# Group by stream and compute message counts and time range
stream_stats = jan_2026.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

# Compute duration in minutes
stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60

# Avoid division by zero
stream_stats = stream_stats[stream_stats["duration_min"] > 0]

# Calculate messages per minute
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Get the stream with the highest messages per minute
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
print("\nStream with the highest messages per minute:")
print(fastest_stream)

# --- Loyalty Check ---
# Get all unique stream IDs in Jan
all_streams = set(jan_2026["stream"].unique())

# Group by user and get the set of streams each user chatted in
user_streams = jan_2026.groupby("user")["stream"].apply(set)

# Filter users who chatted in every stream
active_every_stream = user_streams[user_streams == all_streams]

# Get just the user names
users_in_every_stream = active_every_stream.index.tolist()
print("\nChatters who chatted in every stream in Jan 2026:")
print(users_in_every_stream)

# Longest Stream Calculation
longest_stream = stream_stats.sort_values("duration_min", ascending=False).head(1)

print("\nThe longest stream in Jan 2026 was:")
print(longest_stream[['stream', 'duration_min', 'start_time', 'end_time']])

# Shortest Stream Calculation
shortest_stream = stream_stats.sort_values("duration_min", ascending=True).head(1)

print("\nThe shortest stream in Jan 2026 was:")
print(shortest_stream[['stream', 'duration_min', 'start_time', 'end_time']])

# --- Daily Average Start and End Times (Accounting for Multiple Streams) ---

# 1. Create a "logical date" for each stream. 
# If it starts before 7 AM, it belongs to the previous calendar day's cycle.
stream_stats["logical_date"] = stream_stats["start_time"].apply(
    lambda dt: dt.date() - pd.Timedelta(days=1) if dt.hour < 7 else dt.date()
)

# Convert logical_date to a midnight datetime object so we can do math with it
stream_stats["logical_date_dt"] = pd.to_datetime(stream_stats["logical_date"])

# 2. Group by the logical date to find the earliest start and latest end per day
daily_stats = stream_stats.groupby("logical_date").agg(
    daily_start=("start_time", "min"),
    daily_end=("end_time", "max"),
    logical_date_dt=("logical_date_dt", "first") # Keep this for math
).reset_index()

# 3. Calculate how many total seconds have passed since the midnight of that logical day
# We add .dt.tz_localize(None) to strip the timezone so pandas allows the subtraction
start_secs_from_midnight = (
    daily_stats["daily_start"].dt.tz_localize(None) - daily_stats["logical_date_dt"]
).dt.total_seconds()

end_secs_from_midnight = (
    daily_stats["daily_end"].dt.tz_localize(None) - daily_stats["logical_date_dt"]
).dt.total_seconds()

# 4. Calculate the average seconds
avg_start_sec = start_secs_from_midnight.mean()
avg_end_sec = end_secs_from_midnight.mean()

# 5. Format the seconds back into a readable HH:MM string
def format_avg_time(seconds):
    if pd.isna(seconds): 
        return "N/A"
    # Modulo 24 hours (86400 seconds) to wrap times like 26:00 (2 AM) back to 02:00
    normalized_secs = seconds % (24 * 3600)
    hours = int(normalized_secs // 3600)
    minutes = int((normalized_secs % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

print(f"\nAverage Daily Start Time (Earliest): {format_avg_time(avg_start_sec)}")
print(f"Average Daily End Time (Latest): {format_avg_time(avg_end_sec)}")

Number of new chatters in Jan 2026: 5119
Number of Streams in Jan 2026: 28
Number of Messages in Jan 2026: 245324
Number of Users in Jan 2026: 11641

Top 10 users in Jan 2026:
          user  message_count
0     erdeedge          12388
1    rafa30___          10540
2  polimpompis          10465
3     JBIN2036           9478
4     lajoss__           8257
5     nishad13           8168
6     HALP____           7328
7      Muuskie           7228
8   balintboss           5970
9      Odah_02           4885

Top 10 emotes in Jan 2026:
hi: 9452 times
WW: 6907 times
LOL: 3798 times
omeFaded: 3583 times
ome44: 3560 times
bye: 3328 times
sob: 3252 times
mhm: 3032 times
OMEYES: 2680 times
OOOO: 2627 times

Top 5 busiest 5-minute intervals in Jan 2026:
                          5min  message_count
50   2026-01-03 15:35:00+01:00            618
1409 2026-01-30 16:55:00+01:00            613
1408 2026-01-30 16:50:00+01:00            568
393  2026-01-10 18:30:00+01:00            530
403  2026-01-10 19:2

In [12]:
new_emotes = [
    'ae','doroModCheck','Sup','DOSESHEKNOW','BENNED','bitrate','cortisolspike','mlem','catWait','gkeyWave','paulPls','monkeyPls','mountainlion','Walking','KILLTHATBOY','SenChest','peculiar','Senny','wideSpeedPoint','Emotional','sayWALLAHI','doroPls2','sennyTurkey','paulPiece','SCRAPETHATSHITDORO','doroConcerned','Whatever','noting','doroADHD','DRAIN','Erm','Ermm'
]

final_emote_list = list(set(final_emote_list) | set(new_emotes))

American Arc 04-18

In [13]:
# 1. Define the American Arc window
start_date = "2026-02-04"
end_date = "2026-02-18"

# Filter the main dataframe for the American Arc
# Using .between() is cleaner for specific date ranges
american_arc = data[data["date"].dt.strftime('%Y-%m-%d').between(start_date, end_date)].copy()

# 2. Find the first message date for each user (across the ENTIRE dataset)
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message EVER was during the American Arc
new_chatters = first_messages[
    first_messages["date"].dt.strftime('%Y-%m-%d').between(start_date, end_date)
]

num_new_chatters = new_chatters["user"].nunique()

# --- Summary Stats ---
print(f"--- American Arc Stats ({start_date} to {end_date}) ---")
print(f"Number of new chatters: {num_new_chatters}")

stream_counts = american_arc['stream'].value_counts().reset_index()
print(f"Number of Streams: {len(stream_counts)}")
print(f"Number of Messages: {american_arc.shape[0]}")
print(f"Number of Users: {american_arc['user'].nunique()}")

# --- Top Users ---
user_counts = american_arc['user'].value_counts().reset_index()
user_counts.columns = ['user', 'message_count']
print("\nTop 10 users:")
print(user_counts.head(10))

# --- Emote Analysis ---
emote_counter = Counter()
for message in american_arc["message"]:
    words = str(message).split() # Added str() to prevent errors if any non-strings exist
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

top_10_emotes = emote_counter.most_common(10)
print("\nTop 10 emotes:")
for emote, count in top_10_emotes:
    print(f"{emote}: {count} times")

# --- Activity Spikes ---
american_arc["5min"] = american_arc["date"].dt.floor("5min")
message_counts = american_arc.groupby("5min").size().reset_index(name="message_count")
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("\nTop 5 busiest 5-minute intervals:")
print(top_5_fastest)

# --- Stream Velocity & Duration ---
stream_stats = american_arc.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60
stream_stats = stream_stats[stream_stats["duration_min"] > 0]
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Fastest, Longest, Shortest
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
longest_stream = stream_stats.sort_values("duration_min", ascending=False).head(1)
shortest_stream = stream_stats.sort_values("duration_min", ascending=True).head(1)

print("\nStream with the highest velocity:")
print(fastest_stream[['stream', 'messages_per_min']])

print("\nThe longest stream was:")
print(longest_stream[['stream', 'duration_min', 'start_time', 'end_time']])

# --- Loyalty Check ---
all_arc_streams = set(american_arc["stream"].unique())
user_streams = american_arc.groupby("user")["stream"].apply(set)
active_every_stream = user_streams[user_streams == all_arc_streams]

print(f"\nUsers who attended all {len(all_arc_streams)} streams:")
print(active_every_stream.index.tolist())

# --- Daily Average Times ---
stream_stats["logical_date"] = stream_stats["start_time"].apply(
    lambda dt: dt.date() - pd.Timedelta(days=1) if dt.hour < 7 else dt.date()
)
stream_stats["logical_date_dt"] = pd.to_datetime(stream_stats["logical_date"])

daily_stats = stream_stats.groupby("logical_date").agg(
    daily_start=("start_time", "min"),
    daily_end=("end_time", "max"),
    logical_date_dt=("logical_date_dt", "first")
).reset_index()

# Strip TZs for math
start_secs = (daily_stats["daily_start"].dt.tz_localize(None) - daily_stats["logical_date_dt"]).dt.total_seconds()
end_secs = (daily_stats["daily_end"].dt.tz_localize(None) - daily_stats["logical_date_dt"]).dt.total_seconds()

print(f"\nAverage Daily Start Time: {format_avg_time(start_secs.mean())}")
print(f"Average Daily End Time: {format_avg_time(end_secs.mean())}")

--- American Arc Stats (2026-02-04 to 2026-02-18) ---
Number of new chatters: 1662
Number of Streams: 21
Number of Messages: 105985
Number of Users: 5744

Top 10 users:
           user  message_count
0  FilipStayout           5794
1       Odah_02           4741
2   polimpompis           4551
3      erdeedge           4418
4     rafa30___           4248
5      JBIN2036           3531
6      BenXBari           3501
7    Swimtowin1           2825
8      nishad13           2822
9      lajoss__           2808

Top 10 emotes:
hi: 3430 times
WW: 2900 times
sob: 2589 times
LOL: 2422 times
ome44: 1688 times
mhm: 1579 times
bye: 1427 times
OOOO: 1347 times
LMAO: 1306 times
OMEYES: 1202 times

Top 5 busiest 5-minute intervals:
                         5min  message_count
118 2026-02-06 19:50:00+01:00            518
20  2026-02-05 18:15:00+01:00            505
21  2026-02-05 18:20:00+01:00            500
19  2026-02-05 18:10:00+01:00            444
28  2026-02-05 18:55:00+01:00            444

Str

In [14]:
# 2. Find the first message date for EVERY user in the entire dataset
first_messages = data.groupby("user")["date"].min().reset_index()

# 3. Filter to find "New Chatters" whose FIRST EVER message was during the American Arc
new_chatters = first_messages[
    first_messages["date"].dt.strftime('%Y-%m-%d').between(start_date, end_date)
].copy() # Using .copy() prevents pandas warnings later

# Print total new chatters
num_new_chatters = new_chatters["user"].nunique()
print(f"Number of new chatters during the American Arc: {num_new_chatters}")

# --- 24-Hour Breakdown for New Chatters ---

# 4. Extract the hour of the day (0-23) they sent their first message
new_chatters["hour"] = new_chatters["date"].dt.hour

# 5. Count how many new chatters joined in each hour
hourly_new_chatters = new_chatters.groupby("hour").size().reset_index(name="new_chatters")

# 6. Create a 24-hour template so hours with 0 chatters still show up cleanly
all_hours = pd.DataFrame({"hour": range(24)})
hourly_stats = all_hours.merge(hourly_new_chatters, on="hour", how="left").fillna(0)

# Convert the count to an integer (merge/fillna sometimes turns them into decimals)
hourly_stats["new_chatters"] = hourly_stats["new_chatters"].astype(int)

# 7. Find the absolute best hour
best_hour_row = hourly_stats.sort_values("new_chatters", ascending=False).iloc[0]
best_hour = int(best_hour_row['hour'])
best_count = int(best_hour_row['new_chatters'])

# --- Print the Results ---
print("\n--- Best Hour for New Chatters (American Arc) ---")
print(f"The absolute best hour was {best_hour:02d}:00 with {best_count} new chatters!\n")

print("Full 24-Hour Breakdown:")
for index, row in hourly_stats.iterrows():
    hour_str = f"{int(row['hour']):02d}:00"
    print(f"{hour_str} -> {int(row['new_chatters'])} new chatters")

Number of new chatters during the American Arc: 1662

--- Best Hour for New Chatters (American Arc) ---
The absolute best hour was 19:00 with 242 new chatters!

Full 24-Hour Breakdown:
00:00 -> 160 new chatters
01:00 -> 123 new chatters
02:00 -> 87 new chatters
03:00 -> 36 new chatters
04:00 -> 77 new chatters
05:00 -> 55 new chatters
06:00 -> 26 new chatters
07:00 -> 32 new chatters
08:00 -> 2 new chatters
09:00 -> 0 new chatters
10:00 -> 0 new chatters
11:00 -> 0 new chatters
12:00 -> 0 new chatters
13:00 -> 0 new chatters
14:00 -> 0 new chatters
15:00 -> 0 new chatters
16:00 -> 0 new chatters
17:00 -> 3 new chatters
18:00 -> 183 new chatters
19:00 -> 242 new chatters
20:00 -> 123 new chatters
21:00 -> 122 new chatters
22:00 -> 206 new chatters
23:00 -> 185 new chatters


In [15]:
# Filter only February 2026
feb_2026 = data[(data["date"].dt.year == 2026) & (data["date"].dt.month == 2)]

# Find the first message date for each user
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message was in February 2026
new_chatters = first_messages[
    (first_messages["date"].dt.year == 2026) & (first_messages["date"].dt.month == 2)
]

# Get the number of new chatters
num_new_chatters = new_chatters["user"].nunique()

print(f"Number of new chatters in Feb 2026: {num_new_chatters}")

stream_counts = feb_2026['stream'].value_counts().reset_index()
print(f"Number of Streams in Feb 2026: {len(stream_counts)}")
print(f"Number of Messages in Feb 2026: {feb_2026.shape[0]}")
print(f"Number of Users in Feb 2026: {feb_2026['user'].nunique()}")

# Count number of messages per user in February 2026
user_counts = feb_2026['user'].value_counts().reset_index()

# Rename columns for clarity
user_counts.columns = ['user', 'message_count']

# Get the top 10 users
top_10_users = user_counts.head(10)
print("\nTop 10 users in Feb 2026:")
print(top_10_users)

# --- Emote Analysis ---
emote_counter = Counter()

# Go through each message and count emotes
for message in feb_2026["message"]:
    words = message.split()
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

# Get top 10 emotes
top_10_emotes = emote_counter.most_common(10)

print("\nTop 10 emotes in Feb 2026:")
for emote, count in top_10_emotes:
    print(f"{emote}: {count} times")

# --- Activity Spikes ---
# Make a copy to avoid SettingWithCopyWarning
feb_2026 = feb_2026.copy()

# Round timestamps to nearest 5-minute interval
feb_2026["5min"] = feb_2026["date"].dt.floor("5min")

# Count messages per 5-minute interval
message_counts = feb_2026.groupby("5min").size().reset_index(name="message_count")

# Get top 5 busiest 5-minute intervals
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("\nTop 5 busiest 5-minute intervals in Feb 2026:")
print(top_5_fastest)

# --- Stream Velocity ---
# Group by stream and compute message counts and time range
stream_stats = feb_2026.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

# Compute duration in minutes
stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60

# Avoid division by zero
stream_stats = stream_stats[stream_stats["duration_min"] > 0]

# Calculate messages per minute
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Get the stream with the highest messages per minute
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
print("\nStream with the highest messages per minute:")
print(fastest_stream)

# --- Loyalty Check ---
# Get all unique stream IDs in Feb
all_streams = set(feb_2026["stream"].unique())

# Group by user and get the set of streams each user chatted in
user_streams = feb_2026.groupby("user")["stream"].apply(set)

# Filter users who chatted in every stream
active_every_stream = user_streams[user_streams == all_streams]

# Get just the user names
users_in_every_stream = active_every_stream.index.tolist()
print("\nChatters who chatted in every stream in Feb 2026:")
print(users_in_every_stream)

#Longest Stream Calculation
longest_stream = stream_stats.sort_values("duration_min", ascending=False).head(1)

print("\nThe longest stream in Feb 2026 was:")
print(longest_stream[['stream', 'duration_min', 'start_time', 'end_time']])

#Shortest Stream Calculation
shortest_stream = stream_stats.sort_values("duration_min", ascending=True).head(1)

print("\nThe shortest stream in Feb 2026 was:")
print(shortest_stream[['stream', 'duration_min', 'start_time', 'end_time']])

# --- Daily Average Start and End Times (Accounting for Multiple Streams) ---

# 1. Create a "logical date" for each stream. 
# If it starts before 7 AM, it belongs to the previous calendar day's cycle.
stream_stats["logical_date"] = stream_stats["start_time"].apply(
    lambda dt: dt.date() - pd.Timedelta(days=1) if dt.hour < 7 else dt.date()
)

# Convert logical_date to a midnight datetime object so we can do math with it
stream_stats["logical_date_dt"] = pd.to_datetime(stream_stats["logical_date"])

# 2. Group by the logical date to find the earliest start and latest end per day
daily_stats = stream_stats.groupby("logical_date").agg(
    daily_start=("start_time", "min"),
    daily_end=("end_time", "max"),
    logical_date_dt=("logical_date_dt", "first") # Keep this for math
).reset_index()

# 3. Calculate how many total seconds have passed since the midnight of that logical day
# We add .dt.tz_localize(None) to strip the timezone so pandas allows the subtraction
start_secs_from_midnight = (
    daily_stats["daily_start"].dt.tz_localize(None) - daily_stats["logical_date_dt"]
).dt.total_seconds()

end_secs_from_midnight = (
    daily_stats["daily_end"].dt.tz_localize(None) - daily_stats["logical_date_dt"]
).dt.total_seconds()

# 4. Calculate the average seconds
avg_start_sec = start_secs_from_midnight.mean()
avg_end_sec = end_secs_from_midnight.mean()

# 5. Format the seconds back into a readable HH:MM string
def format_avg_time(seconds):
    if pd.isna(seconds): 
        return "N/A"
    # Modulo 24 hours (86400 seconds) to wrap times like 26:00 (2 AM) back to 02:00
    normalized_secs = seconds % (24 * 3600)
    hours = int(normalized_secs // 3600)
    minutes = int((normalized_secs % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

print(f"\nAverage Daily Start Time (Earliest): {format_avg_time(avg_start_sec)}")
print(f"Average Daily End Time (Latest): {format_avg_time(avg_end_sec)}")

Number of new chatters in Feb 2026: 2637
Number of Streams in Feb 2026: 30
Number of Messages in Feb 2026: 171755
Number of Users in Feb 2026: 7950

Top 10 users in Feb 2026:
           user  message_count
0  FilipStayout           8882
1     rafa30___           8238
2      nishad13           7627
3      erdeedge           7571
4   polimpompis           6939
5       Odah_02           6423
6      JBIN2036           5999
7    Swimtowin1           4550
8    balintboss           4189
9      BenXBari           3872

Top 10 emotes in Feb 2026:
hi: 5404 times
sob: 4168 times
WW: 4060 times
LOL: 3689 times
mhm: 2536 times
LMAO: 2331 times
bye: 2205 times
OMEYES: 2160 times
OOOO: 2136 times
ome44: 1970 times

Top 5 busiest 5-minute intervals in Feb 2026:
                         5min  message_count
185 2026-02-06 19:50:00+01:00            518
87  2026-02-05 18:15:00+01:00            505
88  2026-02-05 18:20:00+01:00            500
86  2026-02-05 18:10:00+01:00            444
95  2026-02-05 18:5